In [ ]:
import os
import pandas as pd
import numpy as np
import datetime
import re
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import shutil

from datetime import datetime
import pytz

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

%matplotlib inline
%config InlineBackend.figure_format='retina'

localtime = pytz.timezone('Europe/London')

# Open and Load the Data

In [ ]:
def process_modular_files(foldersearchpath):
    """
    Process the modular rig files.

    Parameters:
    foldersearchpath (str): The path of the folder containing the modular rig files.

    Returns:
    - df_modular: The processed data from the modular rig files.
    - state_numbers: List of unique state numbers encountered in the files.
    """
    # Step 0: Initialise an empty list to store individual dataframes
    dfs = []

    # Step 0: Initialise an empty list to store unique state numbers
    state_numbers = []

    # Step 1: Define an empty dataframe to store the modular rig data
    df_modular = pd.DataFrame()

    # Step 2: Identify files with "-10s" in the filename
    modular_files = []
    for filename in glob.glob(foldersearchpath + '*-10s.csv'):
        modular_files.append(filename)

    # Step 3: Iterate through modular files
    for filename in modular_files:
        # Step 4: Read the file and select required columns
        df = pd.read_csv(filename, usecols=['runner_timestamp', '4-20mA Sensor 1',
                                            'MFC flow rate', 'Solenoid States', 'volume'])

        # Step 5: Extract state number from filename
        state_number = filename.split("-")[0].split("\\")[-1]

        # Step 6: Check if the state number is already encountered
        if state_number not in state_numbers:
            # Step 7: Add the state number to the list of unique state numbers
            state_numbers.append(state_number)

        # Step 8: Assign state and color based on state number
        df["state_number"] = state_number
        # Step 8: Assign state and color based on state number
        if state_number.startswith('D'):
            df["state"] = "D"
            df["colour"] = "Desorption"
        else:
            df["state"] = "A"
            df["colour"] = "Absorption"

        # Step 9: Rename columns
        df.rename(columns={"runner_timestamp": "Timestamp", "4-20mA Sensor 1": "Pressure (bar)",
                           "MFC flow rate": "Flow (L/min)", "Solenoid States": "Solenoid States",
                           "volume": "Volume (L)"}, inplace=True)

        # Step 10: Set index to runner timestamp and sort chronologically
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df.set_index('Timestamp', inplace=True)
        df.sort_index(inplace=True)

        # Step 11: Add a column for the filename
        df['filename'] = os.path.basename(filename)

        # Step 12: Append dataframe to the list of individual dataframes
        dfs.append(df)

    # Step 13: Concatenate all dataframes in the list
    df_modular = pd.concat(dfs)

    # Step 14: Replace the signal of unplugged sensors, i.e. 0xFF, with NaN values
    df_modular.replace("0xFF", np.NaN, inplace=True)
    df_modular.replace("0xFF ", np.NaN, inplace=True)

    return df_modular, state_numbers

In [ ]:
def process_strain_files(foldersearchpath, state_numbers):
    """
    Process the strain files in the specified folder.

    Parameters:
    - foldersearchpath (str): Path to the folder containing the strain files.
    - state_numbers (list): List of unique state numbers encountered in the modular files.

    Returns:
    - df_strain (DataFrame): DataFrame containing processed strain data with state number as an additional column.
    """
    # Step 0: Initialise an empty list to store individual dataframes
    dfs = []

    # Step 1: Iterate over each state number
    for state_number in state_numbers:
        # Step 2: Identify strain files with the specified state number
        strain_files = glob.glob(foldersearchpath + f'*-{state_number}.csv')

        for filename in strain_files:
            # Step 3: Check if filename contains 4 dashes
            if filename.count('-') != 4:
                continue

            # Step 4: Read the file and skip the first 4 rows
            df = pd.read_csv(filename, skiprows=4, sep=",", usecols=['Timestamp',
                            'M1AI1', 'M2AI0', 'M2AI1', 'M2AI2', 'M2AI3', 'M2AI4', 'M2AI7', 'M3AI0',
                            'M3AI1', 'M3AI2', 'M3AI3', 'M3AI4'], parse_dates=['Timestamp']).set_index('Timestamp')
            df.index = df.index.values.astype(dtype='datetime64')

            # Step 5: Rename columns
            df.rename(columns={"M1AI1": "strain1", "M2AI0": "strain2", "M2AI1": "strain3",
                               "M2AI2": "strain4", "M2AI3": "strain5", "M2AI4": "strain6",
                               "M2AI7": "strain7", "M3AI0": "strain8", "M3AI1": "strain9",
                               "M3AI2": "strain10", "M3AI3": "strain11", "M3AI4": "strain12"}, inplace=True)

            # Step 6: Set the name of the index to 'Timestamp'
            df.index.name = 'Timestamp'

            # Step 7: Convert timestamps in df_strain index to match the format of df_modular
            df.index = pd.to_datetime(df.index).strftime('%Y-%m-%d %H:%M:%S')

            #df['state_number'] = state_number

            # Step 8: Append dataframe to the list
            dfs.append(df)

    # Step 9: Concatenate all dataframes in the list
    df_strain = pd.concat(dfs)

    # Step 10: Convert Timestamp column in df_strain to datetime64[ns]
    df_strain.index = pd.to_datetime(df_strain.index)

    return df_strain

In [ ]:
def process_temperature_files(foldersearchpath, state_numbers):
    """
    Process the temperature files in the specified folder.

    Parameters:
    - foldersearchpath (str): Path to the folder containing the temperature files.
    - state_numbers (list): List of unique state numbers.

    Returns:
    - df_temperature (DataFrame): DataFrame containing the processed temperature data.
    """
    # Step 0: Initialise an empty list to store individual dataframes
    dfs = []

    # Step 1: Iterate through state numbers
    for state_number in state_numbers:
        # Step 2: Find all temperature files for the current state number
        temperature_files = glob.glob(foldersearchpath + f'*-{state_number}.csv') + glob.glob(foldersearchpath + f'*-{state_number}_*.csv')

        # Step 3: Iterate through temperature files
        for filename in temperature_files:
            # Step 4: Check if filename contains 1 dash
            if filename.count('-') != 1:
                continue

            # Step 5: Read the file
            df = pd.read_csv(filename)

            # Step 6: Rename the first column to 'Timestamp'
            df.rename(columns={df.columns[0]: 'Timestamp'}, inplace=True)

            # Step 7: Set the 'Timestamp' column as datetime and format it
            df['Timestamp'] = pd.to_datetime(df['Timestamp']).dt.strftime('%Y-%m-%d %H:%M:%S')

            # Step 8: Set the 'Timestamp' column as the index
            df.set_index('Timestamp', inplace=True)

            # Step 9: Append dataframe to the list
            dfs.append(df)

    # Step 10: Concatenate all dataframes in the list
    df_temperature = pd.concat(dfs)

    # Step 11: Convert Timestamp column in df_temperature to datetime64[ns]
    df_temperature.index = pd.to_datetime(df_temperature.index)

    return df_temperature

In [ ]:
# Define folderpath and reactor_id
folderpath = "C:\\Users\\Sebastiano Gadolini\\Documents\\Python Scripts\\MogRig_DataAnalysis\\"

#reactor_id = "CPE78683"
foldersearchpath = folderpath #+ reactor_id + "\\"

# Call the function to process modular files
df_modular, state_numbers = process_modular_files(foldersearchpath)
# Call the function to process strain files
df_strain = process_strain_files(foldersearchpath, state_numbers)
# Call the function to process temperature files
df_temperature = process_temperature_files(foldersearchpath, state_numbers)

# Drop columns strain10, strain11, and strain12 if not connected
df_strain.drop(columns=['strain10', 'strain11', 'strain12'], inplace=True)

# Dataset Preparation

In [ ]:
def merge_dataframes(df_modular, df_strain, df_temperature):
    """
    Merge the dataframes based on the timestamps in df_modular.

    Parameters:
    - df_modular (DataFrame): DataFrame containing modular rig data.
    - df_strain (DataFrame): DataFrame containing strain data.
    - df_temperature (DataFrame): DataFrame containing temperature data.

    Returns:
    - df_full (DataFrame): Merged DataFrame.
    """
    # Step 0: Sort the indices of df_modular and df_strain
    df_modular.sort_index(inplace=True)
    df_strain.sort_index(inplace=True)
    df_temperature.sort_index(inplace=True)

    # Step 1: Merge df_modular with df_strain on Timestamp
    df_full = pd.merge_asof(df_modular, df_strain, left_index=True, right_index=True, tolerance=pd.Timedelta("1 minute"), direction='nearest')

    # Step 2: Merge the result with df_temperature on Timestamp
    df_full = pd.merge_asof(df_full, df_temperature, left_index=True, right_index=True, tolerance=pd.Timedelta("1 minute"), direction='nearest')

    # Step 3: Replace the signal of unplugged sensors with NaN values
    df_full.replace("0xFF", np.NaN, inplace=True)
    df_full.replace("0xFF ", np.NaN, inplace=True)

    # Step 4: Check if we are in BST/DST, if True, then add an hour to the temp data
    if len(df_temperature) > 0:
        if bool(localtime.localize(df_modular.index[0]).dst()):
            df_temperature.index = df_temperature.index + pd.DateOffset(hours=1)
        df_temperature.sort_index(inplace=True)
    df_full.sort_index(inplace=True)

    return df_full

In [ ]:
# Call the function to merge the dataframes
df_full = merge_dataframes(df_modular, df_strain, df_temperature)

# Data Visualisation

In [ ]:
def flow_rate_chart(df, state_number, plot_filepath=None):
    """
    Generate a scatter plot of flow rate over time.

    Parameters:
    - df (DataFrame): DataFrame containing flow rate data.
    - state_number (str): State number for the plot.
    - folderpath (str, optional): Path to the folder to save the plot. If None, the plot will not be saved.

    Returns:
    None
    """
    fig, ax = plt.subplots(figsize=(6, 6))

    # Define colors for different states
    colors = {'Absorption': 'tab:blue', 'Desorption': 'tab:orange'}

    # Scatter plot of flow rate over time with color based on state
    ax.scatter(df['Time since start (h)'], df['Flow (L/min)'], c=df['colour'].map(colors), marker=".")

    ax.legend()

    ax.set_xlabel("Time Elapsed (h)")
    ax.set_ylabel("Flow (L/min)")

    # Save the plot if plot_filepath is provided
    if plot_filepath:
        plt.savefig(plot_filepath, format='png', bbox_inches='tight')
        plt.close()  # Close the plot to prevent it from being displayed
    else:
        plt.close()


In [ ]:
def flow_rate_temp_chart(df, name, temp_columns, plot_filepath=None):
    """
    Generate a scatter plot of flow rate and temperature over time.

    Parameters:
    - df (DataFrame): DataFrame containing flow rate and temperature data.
    - name (str): Name for the plot.
    - temp_columns (list): List of column names containing temperature data.

    Returns:
    None
    """
    fig, ax = plt.subplots(figsize=(10, 6))
    ax2 = ax.twinx()

    # Scatter plot of flow rate over time
    ax2.scatter(df['Time since start (h)'], df['Flow (L/min)'], color="aqua", marker=".", label=df.iloc[0].state_number + " flow rate")

    # Scatter plot of temperature over time
    for temp in temp_columns:
        ax.scatter(df['Time since start (h)'], df[temp], marker=".", label=temp)

    ax.legend(bbox_to_anchor=(1.1, 1.0), loc='upper left')
    ax2.legend(bbox_to_anchor=(1.1, 0.5), loc='upper left')

    ax.set_xlabel("Time Elapsed (h)")
    ax2.set_ylabel("Flow (L/min)")
    ax.set_ylabel("Temperature (C)")

    plt.tight_layout()

    # Save the plot if plot_filepath is provided
    if plot_filepath:
        plt.savefig(plot_filepath, format='png', bbox_inches='tight')
        plt.close()  # Close the plot to prevent it from being displayed
    else:
        plt.close()

In [ ]:
def volume_chart(df, name, plot_filepath=None):
    """
    Generate a scatter plot of volume over time.

    Parameters:
    - df (DataFrame): DataFrame containing volume data.
    - name (str): Name for the plot.

    Returns:
    None
    """
    fig, ax = plt.subplots(figsize=(6, 6))

    # Define colors for different states
    colors = {'Absorption': 'tab:blue', 'Desorption': 'tab:orange'}

    # Scatter plot of volume over time with color based on state
    ax.scatter(df['Time since start (h)'], df['Volume cumulative sum'], c=df['colour'].map(colors), marker=".", label=df.iloc[0].state_number)

    ax.legend()

    ax.set_xlabel("Time Elapsed (h)")
    ax.set_ylabel("Volume (L)")

    # Save the plot if plot_filepath is provided
    if plot_filepath:
        plt.savefig(plot_filepath, format='png', bbox_inches='tight')
        plt.close()  # Close the plot to prevent it from being displayed
    else:
        plt.close()

In [ ]:
def pressure_chart(df, name, plot_filepath=None):
    """
    Generate a scatter plot of pressure over time.

    Parameters:
    - df (DataFrame): DataFrame containing pressure data.
    - name (str): Name for the plot.

    Returns:
    None
    """
    fig, ax = plt.subplots(figsize=(6, 6))

    # Define colors for different states
    colors = {'Absorption': 'tab:blue', 'Desorption': 'tab:orange'}

    # Scatter plot of pressure over time with color based on state
    ax.scatter(df["Time since start (h)"], df['Pressure (bar)'], c=df['colour'].map(colors), marker=".", label=df.iloc[0].state_number)

    ax.legend()

    ax.set_xlabel("Time Elapsed (h)")
    ax.set_ylabel("Pressure (bar)")

    # Save the plot if plot_filepath is provided
    if plot_filepath:
        plt.savefig(plot_filepath, format='png', bbox_inches='tight')
        plt.close()  # Close the plot to prevent it from being displayed
    else:
        plt.close()

In [ ]:
def strain_pressure_chart(df, name, plot_filepath=None):
    """
    Generate a scatter plot of strain and pressure over time.

    Parameters:
    - df (DataFrame): DataFrame containing strain and pressure data.
    - name (str): Name for the plot.

    Returns:
    None
    """
    fig, ax = plt.subplots(figsize=(6, 6))
    ax2 = ax.twinx()

    # Scatter plot of pressure over time
    ax.scatter(df["Time since start (h)"],
               df['Pressure (bar)'],
               color="aqua",
               marker=".",
               label="_".join(("pressure", df.iloc[0].state_number)))

    # Scatter plot of strain over time
    # IMORTANT if more than 9 strain gages are in place, edit the following to range(1, 13)
    for i in range(1, 10):
        ax2.scatter(df['Time since start (h)'],
                   df["".join(("strain", str(i)))],
                   marker=".",
                   label=("_".join(("strain", str(i), df.iloc[0].state_number))))


    ax2.legend(bbox_to_anchor=(1.2, 1.0), loc='upper left')
    ax.legend(bbox_to_anchor=(1.2, 1.0), loc='lower left')

    ax.set_xlabel("Time Elapsed (h)")
    ax2.set_ylabel("Strain")
    ax.set_ylabel("Pressure (bar)")

    # Save the plot if plot_filepath is provided
    if plot_filepath:
        plt.savefig(plot_filepath, format='png', bbox_inches='tight')
        plt.close()  # Close the plot to prevent it from being displayed
    else:
        plt.close()

In [ ]:
def generate_subsets(df_full):
    """
    Generate subsets of df_full based on unique state numbers.

    Parameters:
    - df_full (DataFrame): DataFrame containing the full data.

    Returns:
    - subsets (dict): Dictionary containing subsets of df_full indexed by state numbers.
    """
    subsets = {}
    state_numbers = df_full['state_number'].unique()

    for state_number in state_numbers:
        df_subset = df_full[df_full['state_number'] == state_number].copy()

        df_subset.sort_index(inplace=True)  # Sort by index (Timestamp)
        df_subset['Time since start (s)'] = (df_subset.index - df_subset.index[0]).total_seconds()
        df_subset['Time since start (h)'] = df_subset['Time since start (s)'] / 3600
        # When the filename is changing reset the time since start to zero and add it to the last value of the previous filename

        prev_filename = None
        time_offset = 0
        for idx, row in df_subset.iterrows():
            if row['filename'] != prev_filename:
                prev_filename = row['filename']
                time_offset = row['Time since start (s)']
            df_subset.at[idx, 'Time since start (s)'] -= time_offset
            df_subset.at[idx, 'Time since start (h)'] = df_subset.at[idx, 'Time since start (s)'] / 3600

        # Calculate volume differences and cumulative sum
        df_subset['vol_diff'] = df_subset['Volume (L)'].diff()
        df_subset['Volume cumulative sum'] = df_subset['vol_diff'].cumsum()

        # Adjust volume cumulative sum for continuity between different filenames
        filenames_sorted = np.sort(df_subset['filename'].unique())
        if len(filenames_sorted) > 1:
            for fn_idx in range(1, len(filenames_sorted)):
                prev_filename = filenames_sorted[fn_idx - 1]
                curr_filename = filenames_sorted[fn_idx]
                prev_last_vol = df_subset[df_subset['filename'] == prev_filename]['Volume (L)'].iloc[-1]
                curr_first_vol = df_subset[df_subset['filename'] == curr_filename]['Volume (L)'].iloc[0]
                df_subset.loc[df_subset['filename'] == curr_filename, 'Volume cumulative sum'] += prev_last_vol
                prev_last_time = df_subset[df_subset['filename'] == prev_filename]['Time since start (s)'].iloc[-1]
                df_subset.loc[df_subset['filename'] == curr_filename, 'Time since start (s)'] += prev_last_time
                df_subset['Time since start (h)'] = df_subset['Time since start (s)'] / 3600

        subsets[state_number] = df_subset

    return subsets

In [ ]:
# Step 1: Generate subsets
subsets = generate_subsets(df_full)

# Step 2: Iterate over subsets and plot charts
for state_number, df_subset in subsets.items():
    flow_rate_chart(df_subset, state_number)
    volume_chart(df_subset, state_number)
    pressure_chart(df_subset, state_number)
    # IMPORTANT: if more than 9 strain gages are in place, edit the following to if {'strain1', 'strain2', 'strain3', 'strain4', 'strain5', 'strain6', 'strain7', 'strain8', 'strain9', 'strain10', 'strain11', 'strain12'}
    if {'strain1', 'strain2', 'strain3', 'strain4', 'strain5', 'strain6', 'strain7', 'strain8', 'strain9'}.issubset(df_full.columns):
        strain_pressure_chart(df_subset, state_number)
    # IMPORTANT: if different thermocouples are in place, edit according to the PicoLog file
    if 'Rig 2 IN Last (C)' in df_full.columns and 'Rig 2 OUT Last (C)' in df_full.columns:
        flow_rate_temp_chart(df_subset, state_number, ['Rig 2 IN Last (C)', 'Rig 2 OUT Last (C)'])

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that 

# Save

In [ ]:
def export_subsets_to_csv_and_plots(subsets, folderpath):
    """
    Export each subset DataFrame to a CSV file with the state number in the filename,
    and save corresponding plots.

    Parameters:
    - subsets (dict): Dictionary of DataFrames indexed by state numbers.
    - folderpath (str): Path to the folder where CSV files and plots will be saved.
    """
    # Create the folder if it doesn't exist
    if not os.path.exists(folderpath):
        os.makedirs(folderpath)

    # Create the "processed raw data" folder if it doesn't exist
    processed_raw_data_folder = os.path.join(folderpath, "processed raw data")
    if not os.path.exists(processed_raw_data_folder):
        os.makedirs(processed_raw_data_folder)

    # Move the CSV files used during the code execution to the "processed raw data" folder
    for filename in os.listdir():
        if filename.endswith("-10s.csv"):
            shutil.move(filename, os.path.join(processed_raw_data_folder, filename))
        elif filename.count("-") == 4 and filename.endswith(".csv"):
            state_number = filename.split("-")[3]
            shutil.move(filename, os.path.join(processed_raw_data_folder, filename))
        elif filename.count("-") == 1 and filename.endswith(".csv"):
            state_number = filename.split("-")[1]
            shutil.move(filename, os.path.join(processed_raw_data_folder, filename))

    # Create the "collated data" folder if it doesn't exist
    collated_data_folder = os.path.join(folderpath, "collated data")
    if not os.path.exists(collated_data_folder):
        os.makedirs(collated_data_folder)

    # Export each subset DataFrame to a CSV file and save plots
    for state_number, df_subset in subsets.items():
        # Define the filename with the state number for CSV
        csv_filename = f"{state_number}_collated.csv"
        # Construct the full file path for CSV
        csv_filepath = os.path.join(collated_data_folder, csv_filename)
        # Export the subset DataFrame to CSV
        df_subset.to_csv(csv_filepath, index=True)

        # Save all the plots
        plot_filename = f"{state_number}_flow_rate_chart.png"
        plot_filepath = os.path.join(collated_data_folder, plot_filename)
        flow_rate_chart(df_subset, state_number, plot_filepath)

        plot_filename = f"{state_number}_volume_chart.png"
        plot_filepath = os.path.join(collated_data_folder, plot_filename)
        volume_chart(df_subset, state_number, plot_filepath)

        plot_filename = f"{state_number}_pressure_chart.png"
        plot_filepath = os.path.join(collated_data_folder, plot_filename)
        pressure_chart(df_subset, state_number, plot_filepath)

        if {'strain1', 'strain2', 'strain3', 'strain4', 'strain5', 'strain6', 'strain7', 'strain8', 'strain9'}.issubset(df_subset.columns):
            plot_filename = f"{state_number}_strain_pressure_chart.png"
            plot_filepath = os.path.join(collated_data_folder, plot_filename)
            strain_pressure_chart(df_subset, state_number, plot_filepath)

        if 'Rig 2 IN Last (C)' in df_subset.columns and 'Rig 2 OUT Last (C)' in df_subset.columns:
            plot_filename = f"{state_number}_flow_rate_temp_chart.png"
            plot_filepath = os.path.join(collated_data_folder, plot_filename)
            flow_rate_temp_chart(df_subset, state_number, ['Rig 2 IN Last (C)', 'Rig 2 OUT Last (C)'], plot_filepath)

In [ ]:
folderpath = "C:\\Users\\Sebastiano Gadolini\\Documents\\Python Scripts\\MogRig_DataAnalysis\\"

export_subsets_to_csv_and_plots(subsets, folderpath)

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that 